In [1]:
from collections import Counter
from math import sqrt
from random import Random
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import re
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import svds
#from scipy.stats.stats import spearmanr
from scipy.stats import spearmanr
from random import shuffle
import math
import random
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
#from sparsesvd import sparsesvd
from numpy.linalg import matrix_rank
from scipy.stats import chi2_contingency
from matplotlib.pyplot import *
from numpy import inf
from scipy.linalg import svd
from foobar import *

In [2]:
glovevector, j, p, q, d, k = loadglovemodelwiki("D:\\20240901paper3\\20240920\\GloVe-master\\GloVe-master\\vectors400.txt", num = 400)
orderedNames = list(glovevector.keys())[:-1]
glovematrix = np.array([glovevector[i] for i in orderedNames])
print(f"Total lines {j}")
print(f"The number is wrong {p}")
print(f"The word is already in the GloVe {q}")
print(f"Error in the line {d}")
print(f"The saved number {k}")

The word is already in the GloVe 2749: 'km'
------------
The word is already in the GloVe 3329: 'million'
------------
The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: -0.317124
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
Error in line 8076:– mm -0.53 '–' due to: could not convert string to float: 'mm'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
------------
The

In [3]:
type(glovematrix)

numpy.ndarray

In [4]:
glovematrix.shape

(15121, 400)

In [5]:
glovevocab, j, p, q, d, k = loadglovevocabwiki("D:\\20240901paper3\\20240920\\GloVe-master\\GloVe-master\\vocab.txt")
print(f"Total lines {j}")
print(f"The number is wrong {p}")
print(f"The word is already in the GloVe {q}")
print(f"Error in the line {d}")
print(f"The saved number {k}")

The word is already in the GloVe 2749: 'km'
------------
The word is already in the GloVe 3329: 'million'
------------
The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: ['63286']
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
The word is already in the GloVe 8076: '–'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
------------
The word is already in the GloVe 8795: 'ft

In [6]:
prices = range(0, glovematrix.shape[0])
vocabsum =  list(glovevocab.values())
vocabtuple = merge(prices, vocabsum)
vocab = dict(zip(glovevocab.keys(), vocabtuple))
glovematrix_vectors_words = fittingmatrixvectors(glovematrix)
glove_vocab = vocab.copy()
svd_vocab = np.load("D:\\20240901paper3\\20240920\\vocabthr10000_dic_index.npy", allow_pickle=True)
svd_vocab = dict(enumerate(svd_vocab.flatten(), 1))[1]

In [7]:
prices

range(0, 15121)

In [8]:
if len(glove_vocab)!=len(svd_vocab):
    print("Not equal")
else:
    flag=0
    for i in glove_vocab:
        if i.isascii():
            if glove_vocab.get(i)[1]!=svd_vocab.get(i)[1]:
                flag=1
                break
    if flag==0:
        print("Both the dictionaries are equal")
    else:
        print("Both the dictionaries are not equal")

Not equal


In [9]:
glovematrix

array([[-0.186779,  0.07054 , -0.0118  , ..., -0.078439, -0.005522,
         0.370395],
       [-0.189564, -0.168155, -0.242808, ...,  0.084276, -0.072041,
         0.362932],
       [ 0.130211, -0.351916,  0.126551, ...,  0.064023,  0.163613,
         0.030938],
       ...,
       [ 0.042206, -0.549411,  0.304823, ..., -0.219849,  0.219528,
        -0.461244],
       [-0.613096,  0.195654, -0.031216, ..., -0.575272,  0.574102,
        -0.567194],
       [-0.517814, -0.135833, -0.016564, ..., -0.849777,  0.120957,
        -0.122439]])

In [10]:
#Evaluate the verctors from GloVe of wiki052024 
dim = ["200", "300", "400", "500", "600"]
test_files = ['wordsim353.txt', 'men_dataset.txt', 'mturk.txt', 'rarewords.txt', 'simlex999.txt']
wiki052024_ws_score_glove = list()
for k in range(len(test_files)):
    print(k)
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[k]
    svd_score = []
    for i in dim:
        print(i)
        #glovevector = loadglovemodel("D:\\Correspondence analysis to PMI-based word embeddings\\GloVe-wiki052024\\vectors"+i+".txt")
        glovevector, j, p, q, d, dd = loadglovemodelwiki("D:\\20240901paper3\\20240920\\GloVe-master\\GloVe-master\\vectors"+i+".txt", num = int(i))
        print(f"Total lines {j}")
        print(f"The number is wrong {p}")
        print(f"The word is already in the GloVe {q}")
        print(f"Error in the line {d}")
        print(f"The saved number {dd}")
        #For Glove, there is extra word (<unk>) vector which is not in vacobulary. Therefore, we delete this word in word vectors: list(glovevector.keys())[-1]
        orderedNames = list(glovevector.keys())[:-1]
        glovematrix = np.array([glovevector[i] for i in orderedNames])
        #glovevocab = loadglovevocab("D:\\Correspondence analysis to PMI-based word embeddings\\GloVe-wiki052024\\vocab.txt")
        glovevocab, j, p, q, d, dd = loadglovevocabwiki("D:\\20240901paper3\\20240920\\GloVe-master\\GloVe-master\\vocab.txt")
        print(f"Total lines {j}")
        print(f"The number is wrong {p}")
        print(f"The word is already in the GloVe {q}")
        print(f"Error in the line {d}")
        print(f"The saved number {dd}")
        prices = range(0, glovematrix.shape[0])
        vocabsum =  list(glovevocab.values())
        vocabtuple = merge(prices, vocabsum)
        vocab = dict(zip(glovevocab.keys(), vocabtuple))

        glovematrix_vectors_words = fittingmatrixvectors(glovematrix)

        svd_score.append(ws_evaluate(glovematrix_vectors_words, vocab, mearsure = 'cosine', path = path_input))
    wiki052024_ws_score_glove.append(svd_score)
np.save("D:\\20240901paper3\\20240920\\wiki052024_ws_score_glove.npy", wiki052024_ws_score_glove)

0
200
The word is already in the GloVe 2749: 'km'
------------
The word is already in the GloVe 3329: 'million'
------------
The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: 0.833260
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
Error in line 8076:– mm -0.20 '–' due to: could not convert string to float: 'mm'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
-----------

The word is already in the GloVe 2749: 'km'
------------
The word is already in the GloVe 3329: 'million'
------------
The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: -0.317124
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
Error in line 8076:– mm -0.53 '–' due to: could not convert string to float: 'mm'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
------------
The

The word is already in the GloVe 2749: 'km'
------------
The word is already in the GloVe 3329: 'million'
------------
The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: -0.206991
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
Error in line 8076:– mm 0.960 '–' due to: could not convert string to float: 'mm'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
------------
The

The word is already in the GloVe 3329: 'million'
------------
The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: -0.441850
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
Error in line 8076:– mm 0.089 '–' due to: could not convert string to float: 'mm'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
------------
The word is already in the GloVe 8795: 'ft'
------------
The

The word is already in the GloVe 2749: 'km'
------------
The word is already in the GloVe 3329: 'million'
------------
The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: 0.580036
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
Error in line 8076:– mm -0.22 '–' due to: could not convert string to float: 'mm'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
------------
The 

...

The word is already in the GloVe 2749: 'km'
------------
The word is already in the GloVe 3329: 'million'
------------
The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: -0.441850
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
Error in line 8076:– mm 0.089 '–' due to: could not convert string to float: 'mm'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
------------
The

500
The word is already in the GloVe 2749: 'km'
------------
The word is already in the GloVe 3329: 'million'
------------
The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: 0.580036
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
Error in line 8076:– mm -0.22 '–' due to: could not convert string to float: 'mm'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
------------


The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: 0.833260
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
Error in line 8076:– mm -0.20 '–' due to: could not convert string to float: 'mm'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
------------
The word is already in the GloVe 8795: 'ft'
------------
The word is already in the GloVe 9695: 'gt'
------------
The word 

The word is already in the GloVe 2749: 'km'
------------
The word is already in the GloVe 3329: 'million'
------------
The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: -0.317124
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
Error in line 8076:– mm -0.53 '–' due to: could not convert string to float: 'mm'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
------------
The

The word is already in the GloVe 2749: 'km'
------------
The word is already in the GloVe 3329: 'million'
------------
The word is already in the GloVe 3425: 'mm'
------------
Number is wrong 4121: -0.206991
------------
The word is already in the GloVe 4574: 'us'
------------
The word is already in the GloVe 4796: 'm'
------------
The word is already in the GloVe 5418: '–'
------------
The word is already in the GloVe 5866: 'gt'
------------
The word is already in the GloVe 6010: 'pm'
------------
The word is already in the GloVe 6552: 'kg'
------------
The word is already in the GloVe 6658: 'sr'
------------
The word is already in the GloVe 7859: 'c'
------------
The word is already in the GloVe 7900: 'cm'
------------
Error in line 8076:– mm 0.960 '–' due to: could not convert string to float: 'mm'
------------
The word is already in the GloVe 8247: 'am'
------------
The word is already in the GloVe 8581: 'ny'
------------
The word is already in the GloVe 8720: 'no'
------------
The

In [17]:
#To look at word pairs in wiki052024

test_files = ['wordsim353.txt', 'men_dataset.txt', 'mturk.txt', 'rarewords.txt', 'simlex999.txt']

wiki052024_test_word_pairs = []

for i in range(len(test_files)):
    
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    
    test_word_pairs_set = read_test_word_pairs_set(path_input)
    
    evaluate_word_pairs_create = evaluate_word_pairs(glovematrix_vectors_words, test_word_pairs_set, glove_vocab, "cosine")
    
    wiki052024_test_word_pairs.append((len(test_word_pairs_set), evaluate_word_pairs_create[1], float(evaluate_word_pairs_create[1]) / len(test_word_pairs_set)))
#table 1
wiki052024_test_word_pairs

[(353, 294, 0.8328611898016998),
 (3000, 2159, 0.7196666666666667),
 (287, 256, 0.89198606271777),
 (2034, 294, 0.14454277286135694),
 (999, 810, 0.8108108108108109)]

In [14]:
wiki052024_ws_score_glove

[[0.6385066483702102,
  0.6555676093690368,
  0.6597624283222892,
  0.6587768843444172,
  0.6633834878442456],
 [0.31174705724406276,
  0.3200736669604224,
  0.32306049774397483,
  0.3260601012435242,
  0.325196288251526],
 [0.6917506478317197,
  0.672803944992707,
  0.6779044438919455,
  0.6750292660887776,
  0.671085061484556],
 [0.5009596719316671,
  0.5161165339134258,
  0.517751699347663,
  0.5095102294341355,
  0.5267041415532465],
 [0.35801952520952046,
  0.36443879163805326,
  0.3766273083560847,
  0.38235293294671024,
  0.38395075803882606]]

In [2]:
0.663+0.326+0.692+0.527+0.384

2.592

In [2]:
wiki052024_ws_score_glove = np.load("D:\\20240901paper3\\20240920\\wiki052024_ws_score_glove.npy")

In [3]:
np.max(wiki052024_ws_score_glove[0]) + np.max(wiki052024_ws_score_glove[1]) + np.max(wiki052024_ws_score_glove[2]) + np.max(wiki052024_ws_score_glove[3]) + np.max(wiki052024_ws_score_glove[4])

2.591849136511562